# import libraries

In [15]:
import numpy as np
import pandas as pd
import math
import pylab as pl
import matplotlib.pyplot as plt
import seaborn as sb
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import chi2_contingency
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from scipy.optimize import curve_fit
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder




# main project

In [16]:
house_price = pd.read_csv('housePrice.csv')

house_price['Area']=pd.to_numeric(house_price['Area'], errors='coerce')

def outliers(df,ft):
    Q1 = df[ft].quantile(0.25)
    Q3 = df[ft].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_boundary = Q1 - 1.5 * IQR
    upper_boundary = Q3 + 1.5 * IQR
    
    lst = df.index[(df[ft] < lower_boundary) | (df[ft] > upper_boundary)]
    return lst

index_list = []
for i in house_price['Area']:
    index_list.extend(outliers(house_price,'Area'))
    
def remove(df,ls):
    ls = sorted(set(ls))
    df = df.drop(ls)
    return df

house_price_cleaned = remove(house_price,index_list)

house_price_cleaned.to_csv('housePrice_cleaned',index=False)

house_price_cleaned.dropna(axis = 0,inplace=True)

house_price_cleaned.reset_index(drop=True,inplace=True)

msk = np.random.rand(len(house_price_cleaned)) < 0.8
train = house_price_cleaned[msk]
test = house_price_cleaned[~msk]

le = LabelEncoder()
le.fit(train['Address'])
train['Address'] = le.transform(train['Address'])
le.fit(test['Address'])
test['Address'] = le.transform(test['Address'])

regr = linear_model.LinearRegression()

train_x = np.asanyarray(train[['Area','Room','Parking','Warehouse','Elevator','Address']])
train_y = np.asanyarray(train[['Price']])
test_x = np.asanyarray(test[['Area','Room','Parking','Warehouse','Elevator','Address']])
test_y = np.asanyarray(test[['Price']])

x = PolynomialFeatures(degree = 2)
train_x_poly = x.fit_transform(train_x)

regr_fit = regr.fit(train_x_poly,train_y)

print(regr_fit.coef_)
print(regr_fit.intercept_)

test_x_poly = x.fit_transform(test_x)
test_y_ = regr_fit.predict(test_x_poly)
print('score is',r2_score(test_y,test_y_))




[[-1.34617721e-06 -6.67047145e+07 -7.31901630e+08 -9.74867838e+08
   3.49062195e+08  6.25112593e+08 -1.98765382e+07  6.22004411e+05
   1.75128875e+06  3.31992460e+07 -2.47689461e+07  1.79668263e+07
  -7.49535952e+04  7.70528892e+07 -1.72888871e+08  5.85862109e+08
  -1.04857455e+09  4.33934403e+06 -9.74867838e+08  6.11022887e+08
   3.61312644e+08  4.98121534e+05  3.49062195e+08  5.49227338e+08
  -4.57992565e+06  6.25112593e+08 -9.84699083e+06  1.59740918e+05]]
[3.64459888e+09]
score is 0.5913037525258383


C:\Users\USER\AppData\Local\Temp\ipykernel_15196\2633665610.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Address'] = le.transform(train['Address'])
C:\Users\USER\AppData\Local\Temp\ipykernel_15196\2633665610.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Address'] = le.transform(test['Address'])
